In [3]:
## importing libraries
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

In [4]:
## import dataset

df = pd.read_csv("data/gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [5]:
## drop id column
df.drop(columns = ['id'],inplace = True)

In [6]:
## segregate the target and features
x = df.drop(columns = ['price'])
y = df['price']

In [7]:
x

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [8]:
y

0         13619
1         13387
2          2772
3           666
4         14453
          ...  
193568     1130
193569     2874
193570     3036
193571      681
193572     2258
Name: price, Length: 193573, dtype: int64

In [11]:
! pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/21/1d/3df58df8bd425f425df9f90b316618ace62b7f1f838ac1580191025cc735/scikit_learn-1.6.0-cp312-cp312-win_amd64.whl.metadata
  Using cached scikit_learn-1.6.0-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/packages/aa/7d/43ab67228ef98c6b5dd42ab386eae2d7877036970a0d7e3dd3eb47a0d530/scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata
  Using cached scipy-1.14.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/91/29/df4b9b42f2be0b623cbd5e2140cafcaa2bef0759a00b7b70104dcfe2fb51/joblib-1.4.2-py3-none-any.whl.metadata
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/4b/2c/ffbf7a134b9ab11a67b0cf0726453cedd9c5043a4fe7


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
## split the data into train-test split
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x, y , test_size = 0.30 , random_state=42)

### Creating Pipeline

In [13]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [14]:
## order of the categories to label them.
cut_labels = ["Fair", "Good", "Ideal", "Very Good", "Premium"]
color_labels = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_lables = ['I1' , 'SI1' , 'SI2' ,'VS1' , 'VS2' , 'VVS1', 'VVS2' , 'IF']

In [15]:
## segregating numeric and categorical columns name
numeric_columns = x.select_dtypes(exclude = 'object').columns
categorical_columns = x.select_dtypes(include = 'object').columns
print(numeric_columns)
print(categorical_columns)

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')
Index(['cut', 'color', 'clarity'], dtype='object')


In [16]:
## automate the preprocessing part (treating missing values, feature encoding and feature scaling) using pipeline.

from sklearn.pipeline import Pipeline

## numerical pipeline
numerical_pipeline = Pipeline( steps = [('imputing', SimpleImputer(strategy='median')), 
                                        ('Scaling' , StandardScaler())
                                       ] )

## categorical pipeline
categorical_pipeline = Pipeline(steps = [('imputing' , SimpleImputer(strategy='most_frequent')),
                              ('encoding' , OrdinalEncoder(categories= [cut_labels , color_labels , clarity_lables])),
                              ('Scaling' , StandardScaler())
                             ] )


## transform the features
preprocessor = ColumnTransformer(
                                [("numeric_pipeline" , numerical_pipeline , numeric_columns),
                                 ("categorical_pipeline" , categorical_pipeline , categorical_columns)])

In [17]:
x.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [18]:
## applying the pipeline to columns
x_train = pd.DataFrame(preprocessor.fit_transform(x_train) , columns= x.columns)
x_test = pd.DataFrame(preprocessor.transform(x_test) , columns = x.columns)

In [19]:
x_train

,carat,cut,color,clarity,depth,table,x,y,z
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,-0.652424,-0.936747,1.799433
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,0.381506,0.910853,0.016413
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,1.415437,0.910853,0.610753
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,-0.652424,-0.320880,1.205093
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,0.381506,1.526720,-1.172267
...,...,...,...,...,...,...,...,...,...
135496,-0.629077,-1.500179,1.964434,-0.546492,-0.518125,-0.644575,0.381506,-0.936747,-1.172267
135497,2.411307,0.443325,2.485700,1.919078,1.872797,1.930288,0.381506,-0.320880,-1.172267
135498,0.923460,0.906065,0.400636,0.992240,0.921862,1.047891,1.415437,0.294987,0.610753
135499,-1.038774,-0.667249,-0.641897,-1.212375,-1.197364,-1.252127,0.381506,0.294987,1.205093


In [20]:
x_test

,carat,cut,color,clarity,depth,table,x,y,z
0,-0.629077,0.258230,-0.120630,-0.600482,-0.581521,-0.572248,-0.652424,-1.552614,-1.172267
1,2.605374,-2.148014,-0.120630,2.126042,2.198832,1.959219,0.381506,0.294987,-0.577927
2,-1.125026,-1.222536,0.921902,-1.374347,-1.414721,-1.469110,1.415437,-0.936747,1.205093
3,-1.017211,-0.574701,0.921902,-1.158385,-1.161138,-1.194265,1.415437,1.526720,1.205093
4,0.858771,0.628421,-0.641897,0.947248,0.985258,1.004495,-0.652424,0.910853,-1.172267
...,...,...,...,...,...,...,...,...,...
58067,0.255007,0.535873,0.921902,0.416340,0.369414,0.425874,0.381506,1.526720,-0.577927
58068,-0.607514,0.535873,-0.641897,-0.528495,-0.554351,-0.499920,-0.652424,-1.552614,0.610753
58069,-0.823144,-0.019414,-0.641897,-0.834441,-0.862273,-0.847093,-0.652424,0.294987,1.205093
58070,0.901897,-0.667249,1.443168,1.046230,0.967145,0.932167,1.415437,1.526720,-1.172267


In [21]:
## importing models

from sklearn.linear_model import LinearRegression , Lasso , Ridge , ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
#from sklearn.neighbors import KNeighborsRegressor
#from sklearn.svm import SVR

## evaluation metrics
from sklearn.metrics import mean_absolute_error , mean_squared_error , r2_score

In [22]:
def model_evaluation(model , x_train , y_train , x_test , y_test):
    model.fit(x_train , y_train)
    y_pred = model.predict(x_test)

    mae = mean_absolute_error(y_test , y_pred)
    mse = mean_squared_error(y_test , y_pred)
    rmse = np.sqrt(mse)
    r_score = r2_score(y_test , y_pred)
    
    return mae,mse,rmse,r_score

In [52]:
models = {"LinearRegression" : LinearRegression(),
         "Lasso" : Lasso(),
         "ElasticNet" : ElasticNet(),
        #  "DTRegressor" : DecisionTreeRegressor(),
        #  "RFRegressor" : RandomForestRegressor()
         }

In [51]:
evaluation_metric = []   # empty list to store each model performance
for model in models :
    mae,mse,rmse,r_score = model_evaluation(models[model] , x_train , y_train , x_test , y_test)
    metrics = {"model" : model,
             "mae" : mae,
             "mse" : mse,
             "rmse": rmse,
             "r_score": r_score}
    evaluation_metric.append(metrics)

KeyboardInterrupt: 

In [24]:
evaluation_metric

[{'model': 'LinearRegression',
  'mae': 704.417811142695,
  'mse': 1171420.932249337,
  'rmse': np.float64(1082.3220095005631),
  'r_score': 0.9275062064898284},
 {'model': 'Lasso',
  'mae': 705.1671346510686,
  'mse': 1171466.78135067,
  'rmse': np.float64(1082.3431901900017),
  'r_score': 0.9275033691021797},
 {'model': 'ElasticNet',
  'mae': 1075.8686451148828,
  'mse': 2441380.566139904,
  'rmse': np.float64(1562.4917811431533),
  'r_score': 0.8489143110140183},
 {'model': 'DTRegressor',
  'mae': 427.32932968269273,
  'mse': 717034.2701823214,
  'rmse': np.float64(846.7787610600076),
  'r_score': 0.9556260837660618},
 {'model': 'RFRegressor',
  'mae': 309.9620116588986,
  'mse': 374009.43607354356,
  'rmse': np.float64(611.5631088232379),
  'r_score': 0.9768542954260612}]

In [25]:
## cross validation
from sklearn.model_selection import cross_val_score

In [63]:

def best_model(models_dict , x_train, y_train):
    model_Scores = []
    for model in models:
        score = cross_val_score(models[model], X=x_train , y=y_train , cv=5,scoring='r2').mean()
        model_Scores.append((model,score))

        print(model ,"->" ,"R2_score : " , score)
        print("="*30 ,'\n')


    model_Scores.sort(key=lambda x:x[1])
    best_model_score = model_Scores[-1]

    return best_model_score

In [64]:
best_model_value = best_model(models_dict = models , x_train=x_train , y_train=y_train)
print(best_model_value[0] , best_model_value[1])

LinearRegression -> R2_score :  0.9279295881007952

Lasso -> R2_score :  0.9279901180428608

ElasticNet -> R2_score :  0.8493352737039646

Lasso 0.9279901180428608


In [67]:
## accessing best model
models[best_model_value[0]]

Lasso()

In [ ]:
## create the pipeline for these three steps
# feature encoding
# train test split
# feature scaling

# model building
# model evaluation
# model selection